# Support code for Mads Olsen's group's model

In [1]:
#| default_exp mads_olsen_support

In [2]:
#| hide 
%load_ext autoreload
%autoreload 2

In [3]:
#| hide
from nbdev.showdoc import *


In [4]:
#| export

from typing import Dict, List, Tuple

from pathlib import Path

import numpy as np

In [5]:
#| export

"""
This section is copied straight from https://github.com/MADSOLSEN/SleepStagePrediction, pulling just the functions needed for the classifier and modifying some of them to work here.
"""

import sys
from functools import partial

import numpy as np
import pandas as pd
import polars as pl
from scipy.signal import spectrogram

from functools import partial

import tensorflow as tf
import pkg_resources

def median(x, fs, window_size):
    if isinstance(x, pl.DataFrame):
        x = x.to_numpy()

    window = (
        (fs * window_size + 1) if (fs * window_size) % 2 == 0 else (fs * window_size)
    )

    reduce_dims = False
    if len(x.shape) == 1:
        x = np.expand_dims(x, axis=-1)
        reduce_dims = True
    x_norm = np.zeros((x.shape))

    for idx in range(x.shape[-1]):

        x_med = np.ones((x.shape[0])) * np.median(x[:, idx])

        x_pd = pd.Series(x[:, idx])
        med_ = x_pd.rolling(window).median()
        x_med[int(window / 2) : -int(window / 2)] = med_[window - 1 :]
        x_med[: int(window / 2)] = med_[window - 1]
        x_med[-int(window / 2) :] = med_[-1:]

        x_med[np.isnan(x_med)] = 0  # remove nan

        x_norm[:, idx] = x[:, idx] - x_med

    if reduce_dims:
        x_norm = x_norm[:, 0]
    return x_norm


def clip_by_iqr(x, fs, threshold=20):

    x[x > threshold] = threshold
    x[x < -threshold] = -threshold

    return x

def iqr_normalization_adaptive(x, fs, median_window, iqr_window):
    def normalize(x, fs, median_window, iqr_window, iqr_upper=0.75, iqr_lower=0.25):

        # add noise
        x_ = x + np.random.normal(loc=0, scale=sys.float_info.epsilon, size=(x.shape))

        # fix window parameters to odd number
        med_window = (
            (fs * median_window + 1)
            if (fs * median_window) % 2 == 0
            else (fs * median_window)
        )
        iqr_window = (
            (fs * iqr_window + 1) if (fs * iqr_window) % 2 == 0 else (fs * iqr_window)
        )

        # preallocation
        x_med = np.ones((x.shape)) * np.median(x_)
        x_iqr_up = np.ones((x.shape)) * np.quantile(x_, iqr_upper)
        x_iqr_lo = np.ones((x.shape)) * np.quantile(x_, iqr_lower)

        # find rolling median
        x_pd = pd.Series(x_)
        med_ = x_pd.rolling(med_window).median()
        x_med[int(med_window / 2) : -int(med_window / 2)] = med_[med_window - 1 :]
        x_med[np.isnan(x_med)] = 0  # remove nan

        # find rolling quantiles
        x_iqr_upper = x_pd.rolling(iqr_window).quantile(iqr_upper)
        x_iqr_lower = x_pd.rolling(iqr_window).quantile(iqr_lower)

        # border padding
        x_iqr_up[int(iqr_window / 2) : -int(iqr_window / 2)] = x_iqr_upper[
            iqr_window - 1 :
        ]
        x_iqr_lo[int(iqr_window / 2) : -int(iqr_window / 2)] = x_iqr_lower[
            iqr_window - 1 :
        ]

        # remove nan
        x_iqr_up[np.isnan(x_iqr_up)] = 0
        x_iqr_lo[np.isnan(x_iqr_lo)] = 0

        # return normalize
        return (x_ - x_iqr_lo) / (x_iqr_up - x_iqr_lo + sys.float_info.epsilon) * 2 - 1

    x_norm = np.zeros((x.shape))
    if len(x.shape) == 1:
        x_norm[:] = normalize(x, fs, median_window, iqr_window)
    else:
        for n in range(x.shape[1]):
            x_norm[:, n] = normalize(x[:, n], fs, median_window, iqr_window)
    return x_norm



def cal_psd(x, fs, window, noverlap, nfft, f_min, f_max, f_sub=1):
    """
    https://github.com/MADSOLSEN/SleepStagePrediction/blob/d47ff488f5cedd3b0459593a53fc4f92fc3660a2/signal_processing/spectrogram.py#L91
    """
    from scipy.ndimage import maximum_filter as maxfilt

    # border edit
    x_ = np.zeros((x.size + window,))
    x_[window // 2 : -window // 2] = x
    x_ = x_ + np.random.normal(loc=0, scale=sys.float_info.epsilon, size=(x_.shape))

    f, t, S = spectrogram(
        x=x_,
        fs=fs,
        window=np.blackman(window),
        nperseg=window,
        noverlap=noverlap,
        nfft=nfft,
    )

    S = S[(f > f_min) & (f <= f_max), :]
    S = maxfilt(np.abs(S), size=(f_sub, 1))
    S = S[::f_sub, :]
    S = np.swapaxes(S, axis1=1, axis2=0)
    S = np.log(S + sys.float_info.epsilon)
    if S.shape[1] != 24:
        k = 1

    assert np.sum(np.isinf(S)) == 0
    assert np.sum(np.isnan(S)) == 0
    return S

def load_saved_keras():
    file_path = pkg_resources.resource_filename('pisces', 'cached_models/mo_resunet.keras')

    return tf.keras.models.load_model(file_path, safe_mode=False)


In [6]:
#| export

FS = 32
CHANNELS = 2
DEPTH = 9
N_OUT = 2 ** (DEPTH + 1)
N_CLASSES = 4

MO_UNET_CONFIG = {
    "input_shape": (15360, 32, CHANNELS),
    "num_classes": N_CLASSES,
    "num_outputs": N_OUT,
    "init_filter_num": 8,  # 16,
    "filter_increment_factor": 2 ** (1 / 3),
    "max_pool_size": (2, 2),
    "depth": DEPTH,
    "kernel_size": (16, 3),
}

MO_PREPROCESSING_CONFIG = {
    "preprocessing": [
        {"args": {"window_size": 30, "fs": FS}, "type": "median"},
        {
            "args": {"iqr_window": 300, "median_window": 300, "fs": FS},
            "type": "iqr_normalization_adaptive",
        },
        {"args": {"threshold": 20, "fs": FS}, "type": "clip_by_iqr"},
        {
            "args": {
                "fs": FS,
                "nfft": 512,
                "f_max": 6,
                "f_min": 0,
                "f_sub": 3,
                "window": 320,
                "noverlap": 256,
            },
            "type": "cal_psd",
        },
    ]
}

In [7]:
#| export 

from keras import layers
from keras.models import Model
from keras_cv.layers import StochasticDepth
import numpy as np
from keras.regularizers import l2

def factory_ResUNet(input_shape,
            num_classes,
            num_outputs,
            depth=None,
            init_filter_num=8,
            filter_increment_factor=2 ** (1 / 3),
            kernel_size=(16, 1),
            max_pool_size=(2, 1),
            activation='gelu',
            output_layer='sigmoid',
            weight_decay=0.0,
            residual=False,
            stochastic_depth=False,
            data_format = 'channels_last') -> Model:


    if depth is None:
        depth = determine_depth(temporal_shape=input_shape[0], temporal_max_pool_size=max_pool_size[0])

    x_input = layers.Input(shape=tuple(input_shape))
    x = x_input

    # zero-pad:
    zeros_to_add = int(2 ** (np.ceil(np.log2(input_shape[0]))) - input_shape[0])
    if (zeros_to_add > 0) and (zeros_to_add / 2 == zeros_to_add // 2):
        x = layers.ZeroPadding2D(padding=(zeros_to_add // 2, 0))(x)

    # preallocation
    features = init_filter_num
    skips = []
    features_list = []
    kernel_size_list = []
    max_pool_size_list = []


    # Encoder
    # ========================================================
    for i in range(depth):

        # append lists of variables:
        features_list.append(features)
        kernel_size_list.append(kernel_size)
        max_pool_size_list.append(max_pool_size)

        # Feature extractor
        x = conv_block(x=x, features=int(features), kernel_size=kernel_size, activation=activation,
                       data_format=data_format, weight_decay=weight_decay, residual=residual, stochastic_depth=stochastic_depth)
        skips.append(x)
        features *= filter_increment_factor

        # Reshape output to subsequent layer:
        x = layers.Conv2D(int(features),
                          kernel_size=max_pool_size,
                          activation=None,
                          padding='same',
                          strides=max_pool_size,
                          data_format=data_format,
                          kernel_regularizer=l2(weight_decay),
                          bias_regularizer=l2(weight_decay))(x)
        x = layers.BatchNormalization()(x)

        # update kernel_size and max_pool_size
        kernel_size = [min(ks, x_dim) for ks, x_dim in zip(kernel_size, x.shape[1:3])]
        if x.shape[2] / max_pool_size[1] < 1:
            max_pool_size = (max_pool_size[0], 1)


    # Middel part
    # ========================================================
    x = conv_block(x=x, features=int(features), kernel_size=kernel_size, activation=activation,
                       data_format=data_format, weight_decay=weight_decay, residual=residual)


    # Decoder
    # ========================================================
    features_list = [int(f) for f in features_list]
    for count, i in enumerate(reversed(range(depth))):

        # upsample and convolve
        x = layers.Conv2DTranspose(features_list[i],
                                   kernel_size=[int(mp) for mp in max_pool_size_list[i]],
                                   strides=[int(mp) for mp in max_pool_size_list[i]],
                                   padding='same',
                                   activation=None,
                                   data_format=data_format)(x)
        x = layers.BatchNormalization()(x)

        # concatenate with layer from encoder with same dimensionality
        x = layers.concatenate([skips[i], x], axis=3)

        # feature extractor
        x = conv_block(x=x, features=features_list[i], kernel_size=kernel_size_list[i], activation=activation,
                       data_format=data_format, weight_decay=weight_decay, residual=residual, stochastic_depth=stochastic_depth)


    # Cut-off zero-padded segment:
    if (zeros_to_add > 0) and (zeros_to_add / 2 == zeros_to_add // 2):
        x = layers.Lambda(lambda z: z[:, zeros_to_add // 2: - zeros_to_add // 2, :, :])(x)

    # reshape
    x = layers.Reshape((x.shape[1], x.shape[2] * x.shape[3]))(x)

    # non-linear activation:
    x = layers.Conv1D(filters=init_filter_num ,
                      kernel_size=1,
                      padding='same',
                      activation=activation,
                      kernel_regularizer=l2(weight_decay),
                      bias_regularizer=l2(weight_decay)
                      )(x)

    if input_shape[0] // num_outputs > 0:
        x = layers.AveragePooling1D(pool_size=input_shape[0] // num_outputs)(x)


    # non-linear activation:
    x = layers.Conv1D(filters=num_classes,
                  kernel_size=1,
                  padding='same',
                  activation=activation,
                  kernel_regularizer=l2(weight_decay),
                  bias_regularizer=l2(weight_decay)
                  )(x)


    # Classification
    # ========================================================
    x = layers.Dense(units=num_classes,
                     activation=output_layer)(x)

    return Model(inputs=x_input, outputs=x)


def conv_block(x, features, kernel_size, data_format='channels_last', weight_decay=0.0,
               residual=True, stochastic_depth=True, activation='gelu'):

    # feature extractor
    x_ = layers.Conv2D(int(features),
                       kernel_size=kernel_size,
                       activation=activation,
                       padding='same',
                       data_format=data_format,
                       kernel_regularizer=l2(weight_decay),
                       bias_regularizer=l2(weight_decay))(x)
    x_ = layers.BatchNormalization()(x_)

    if residual:
        if x.shape[-1] != x_.shape[-1]:
            x = layers.Conv2D(int(features),
                              kernel_size=(1, 1),
                              activation=None,
                              padding='same',
                              data_format=data_format,
                              kernel_regularizer=l2(weight_decay),
                              bias_regularizer=l2(weight_decay))(x)
        if stochastic_depth:
            return StochasticDepth(survival_probability=0.9)([x, x_])
        else:
            return layers.Add()([x, x_])
    else:
        return x_


def determine_depth(temporal_shape, temporal_max_pool_size):

    depth = 0
    while temporal_shape % 2 == 0:
        depth += 1
        temporal_shape /= round(temporal_max_pool_size)
    depth -= 1
    return depth



In [8]:

from dataclasses import dataclass
events = ['wake', 'light', 'deep', 'rem']
events_format = [
    {
        'name': 'wake', 
        'h5_path': 'wake',
        'probability': 1 / len(events)
    },
    {
        'name': 'light', 
        'h5_path': 'light',
        'probability': 1 / len(events)
    },
    {
        'name': 'deep', 
        'h5_path': 'deep',
        'probability': 1 / len(events)
    },
    {
        'name': 'rem', 
        'h5_path': 'rem',
        'probability': 1 / len(events)
    }
]
signals_format =  {
    "ACC_merge_fft_spec": {
        "add": True,
        "fs_post": 0.5,
        "h5_path": "acc_signal",
        "dimensions": [
            32,
            1
        ],
        "channel_idx": [
            0,
            1,
            2
        ],
        "preprocessing": [
            {
                "args": {
                    "window_size": 30
                },
                "type": "median"
            },
            {
                "args": {
                    "iqr_window": 300,
                    "median_window": 300
                },
                "type": "iqr_normalization_adaptive"
            },
            {
                "args": {
                    "threshold": 20
                },
                "type": "clip_by_iqr"
            },
            {
                "args": {
                    "nfft": 512,
                    "f_max": 6,
                    "f_min": 0,
                    "f_sub": 3,
                    "window": 320,
                    "noverlap": 256
                },
                "type": "cal_psd"
            }
        ],
        "transformations": {
            "freq_mask": {},
            "time_mask": {},
            "image_translation": {}
        },
        "batch_normalization": {}
    },
    "PPG_fft_spec": {
        "add": False,
        "fs_post": 0.5,
        "h5_path": "ppg_signal",
        "dimensions": [
            32,
            1
        ],
        "channel_idx": [
            0
        ],
        "preprocessing": [
            {
                "args": {},
                "type": "zscore"
            },
            {
                "args": {},
                "type": "change_PPG_direction"
            },
            {
                "args": {
                    "iqr_window": 301,
                    "median_window": 301
                },
                "type": "iqr_normalization_adaptive"
            },
            {
                "args": {
                    "threshold": 20
                },
                "type": "clip_by_iqr"
            },
            {
                "args": {
                    "nfft": 512,
                    "f_max": 2.1,
                    "f_min": 0.1,
                    "f_sub": 1,
                    "window": 320,
                    "noverlap": 256
                },
                "type": "cal_psd"
            }
        ],
        "transformations": {
            "freq_mask": {},
            "time_mask": {},
            "image_translation": {}
        },
        "batch_normalization": {}
    }
}
data_directory = '.'

dataset_params = {
    "h5_directory": data_directory, 
    "signals_format": signals_format,
    "window": 30 * 2 ** 10, 
    "number_of_channels": len(signals_format), 
    "events_format": events_format,
    "prediction_resolution": 30,
    "overlap": 0.25,
    "minimum_overlap": 0.1,
    "batch_size": 2,
    "cache_data": True,
    "n_jobs": 4,
    "use_mask": True,
    "load_signal_in_RAM": True
}

@dataclass
class DSLite:
    fsTime: float
    nSpace: int
    nChannels: int
    window: int
    prediction_resolution: int

    def __init__(self, h5_directory, signals_format, window, overlap, batch_size, minimum_overlap, events_format, number_of_channels, prediction_resolution, load_signal_in_RAM, use_mask, cache_data, n_jobs):
# datasets
        self.h5_directory = h5_directory

        # signal modalities
        self.signals_format = signals_format
        self.window = window
        self.number_of_channels = number_of_channels
        self.prediction_resolution = prediction_resolution
        self.overlap = overlap
        self.batch_size = batch_size
        self.predictions_per_window = window // prediction_resolution
        self.nChannels = sum([sf['dimensions'][-1] for sf in signals_format.values()])
        self.nSpace = [sf['dimensions'][0] for sf in signals_format.values()][0] # assumes same space resolution
        self.fsTime = [sf['fs_post'] for sf in signals_format.values()][0] # assumes same temporal resolution

        # events
        self.events_format = events_format
        self.minimum_overlap = minimum_overlap
        self.number_of_events = len(events_format)
        self.number_of_classes = len(events_format)
        self.event_probabilities = [event['probability'] for event in events_format]
        self.event_labels = [event['name'] for event in events_format]
        assert sum(self.event_probabilities) <= 1

        # training
        self.load_signal_in_RAM = load_signal_in_RAM
        self.use_mask = use_mask



In [9]:
import sys

ds_train = DSLite(**dataset_params)

# model creation
model_params = {
    'input_shape': [int(ds_train.fsTime * ds_train.window), ds_train.nSpace, ds_train.nChannels], 
    'num_classes': len(events),
    'num_outputs': ds_train.window // ds_train.prediction_resolution,
    'depth': 9,
    'init_filter_num': 16,
    'filter_increment_factor': 2 ** (1 / 3),
    'max_pool_size': (2, 2),
    'kernel_size': (16, 3)
}

resunet = factory_ResUNet(**model_params)
resunet.summary() # print summary.

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 15360, 32, │          0 │ -                 │
│ (InputLayer)        │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 16384, 32, │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 16384, 32, │      1,552 │ zero_padding2d[0… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 16384, 32, │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 8192, 16,  │      1,300 │ batch_normalizat… │
│                     │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8192, 16,  │         80 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 8192, 16,  │     19,220 │ batch_normalizat… │
│                     │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8192, 16,  │         80 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 4096, 8,   │      2,025 │ batch_normalizat… │
│                     │ 25)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4096, 8,   │        100 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 25)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 4096, 8,   │     30,025 │ batch_normalizat… │
│                     │ 25)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4096, 8,   │        100 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 25)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 2048, 4,   │      3,232 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2048, 4,   │        128 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 2048, 4,   │     49,184 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2048, 4,   │        128 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 1024, 2,   │      5,160 │ batch_normalizat

 Total params: 2,024,936 (7.72 MB)

 Trainable params: 2,021,032 (7.71 MB)

 Non-trainable params: 3,904 (15.25 KB)

In [10]:
#| hide
import nbdev
nbdev.nbdev_export()